In [1]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from smogn import smoter
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_percentage_error
import optuna
import warnings

# Set Optuna's logging level to WARNING to suppress informational logs
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Suppress specific warnings related to Optuna's distribution
warnings.filterwarnings(
    "ignore",
    message="The distribution is specified by.*and step=.*but the range is not divisible by `step`.*",
)

In [4]:
import geopandas as gpd
import pandas as pd
import numpy as np
from functools import partial
import random

import joblib

sample_cities = gpd.read_feather("closest_cities.feather")
cities = gpd.read_parquet(
    "/Users/test/Documents/code/IL2/factory_loc_service/api/app/data/cities.parquet"
)

In [5]:
random.seed(7575)
np.random.seed(7575)

In [6]:
cols = [
    "population",
    "harsh_climate",
    # "ueqi_score",
    "ueqi_residential",
    "ueqi_street_networks",
    "ueqi_green_spaces",
    "ueqi_public_and_business_infrastructure",
    "ueqi_social_and_leisure_infrastructure",
    "ueqi_citywide_space",
    "median_salary",
    "factories_total",
]

In [7]:
def _preprocess_x(cities, scaler_x, fit=False):
    X = cities[cols]

    # Step 1: Initialize the MinMaxScaler
    if fit:
        X_scaled = scaler_x.fit_transform(X)
        joblib.dump(scaler_x, "scaler_x.pkl")
    else:
        X_scaled = scaler_x.transform(X)
    return X_scaled


# def preprocess_y(cities):
#     return np.log(cities["num_in_migration"])


def _resample(X_scaled, y):
    # Создаем сбалансированный датасет для регрессии
    X_resampled = smoter(
        data=pd.concat([pd.DataFrame(X_scaled), y.to_frame()], axis=1).dropna(),
        y="num_in_migration",
        k=140,
        pert=0.03,
        rel_thres=0.3,  # порог для определения редких значений
        rel_method="auto",  # автоматическое определение редких значений
        samp_method="extreme",
    )

    # Разделяем признаки и целевую переменную
    y_resampled = np.log(X_resampled["num_in_migration"] + 1)
    X_resampled = np.log(X_resampled.drop("num_in_migration", axis=1) + 1)

    return X_resampled, y_resampled


def _define_model():
    return CatBoostRegressor(
        loss_function="Quantile",  # Consider 'Tweedie' if suitable for your data with 'variance_power'
        eval_metric="MAPE",
        iterations=3000,  # Reduce iterations when increasing learning_rate
        learning_rate=0.01,  # Increase for faster learning, adjusted with iterations
        depth=8,  # Reduce depth for better generalization
        random_seed=42,
        early_stopping_rounds=100,  # Allow more rounds for potential convergence
        l2_leaf_reg=3.0,  # Regularization to prevent overfitting
        subsample=0.7,  # Sample 80% of data to introduce randomness
        random_strength=2.0,  # Controls feature splits for regularization
        border_count=128,
    )


def _split(X_resampled, y_resampled):
    return train_test_split(X_resampled, y_resampled, test_size=0.3, random_state=42)


def _calculate_weights(y):
    # Создаем веса обратно пропорциональные значениям
    weights = 1 / (1 + np.log1p(y))
    # Нормализуем веса
    weights = weights / weights.mean()
    return weights


def make_regr_model(cities):

    # old_all_cities = cities.copy()
    cities = cities[
        (cities["num_in_migration"] > 1)
        & (cities["num_in_migration"] < 500)
        # & (cities["rel_migr"] > 0)
        & (cities["population"] < 1e6)
    ].copy()

    scaler_x = MinMaxScaler()
    model = _define_model()

    X_scaled = _preprocess_x(cities, scaler_x, fit=True)

    # X = _preprocess_x(old_all_cities, fit=True)
    # Y = old_all_cities["num_in_migration"].reset_index(drop=True)
    y = cities["num_in_migration"].reset_index(drop=True)

    X_resampled, y_resampled = _resample(X_scaled, y)

    X_train, X_test, y_train, y_test = _split(X_resampled, y_resampled)

    # Создаем веса
    sample_weights = _calculate_weights(y_train[:-50])
    # Обучаем модель с весами
    # model = CatBoostRegressor(loss_function="MAE", iterations=1000)

    model.fit(
        X_train.iloc[:-50],
        y_train.iloc[:-50],
        sample_weight=sample_weights,
        eval_set=(X_train.iloc[-50:], y_train.iloc[-50:]),
        early_stopping_rounds=50,
        verbose=300,
    )

    y_test = np.exp(y_test)
    y_train = np.exp(y_train)

    pred_test = np.exp(model.predict(X_test))
    print(mean_absolute_percentage_error(pred_test, y_test))

    pred_train = np.exp(model.predict(X_train))
    print(mean_absolute_percentage_error(pred_train, y_train))

    pred_train = np.exp(model.predict(X_resampled))
    print(mean_absolute_percentage_error(pred_train, np.exp(y_resampled)))

    return model, X_resampled, X_train, X_test, y_train, y_test, scaler_x

In [8]:
def objective(
    trial,
    # target_migration,
    population,
    harsh_climate,
    ueqi_residential_current,
    ueqi_street_networks_current,
    ueqi_green_spaces_current,
    ueqi_public_and_business_infrastructure_current,
    ueqi_social_and_leisure_infrastructure_current,
    ueqi_citywide_space_current,
    median_salary_current,
    factories_total,
    model,
):
    # Sample parameters using Optuna's suggested distributions
    # population = trial.suggest_float("population", min_pop, max_pop)
    ueqi_residential = trial.suggest_float(
        "ueqi_residential", ueqi_residential_current, 1, step=1e-3
    )
    ueqi_street_networks = trial.suggest_float(
        "ueqi_street_networks", ueqi_street_networks_current, 1, step=1e-3
    )
    ueqi_green_spaces = trial.suggest_float(
        "ueqi_green_spaces", ueqi_green_spaces_current, 1, step=1e-3
    )
    ueqi_public_and_business_infrastructure = trial.suggest_float(
        "ueqi_public_and_business_infrastructure",
        ueqi_public_and_business_infrastructure_current,
        1,
        step=1e-3,
    )
    ueqi_social_and_leisure_infrastructure = trial.suggest_float(
        "ueqi_social_and_leisure_infrastructure",
        ueqi_social_and_leisure_infrastructure_current,
        1,
        step=1e-3,
    )
    ueqi_citywide_space = trial.suggest_float(
        "ueqi_citywide_space", ueqi_citywide_space_current, 1, step=1e-3
    )

    median_salary = trial.suggest_float(
        "median_salary", median_salary_current, 1, step=1e-3
    )

    # Add more parameters if needed

    # Combine the parameters into a feature vector
    city_parameters = [
        population,
        harsh_climate,
        ueqi_residential,
        ueqi_street_networks,
        ueqi_green_spaces,
        ueqi_public_and_business_infrastructure,
        ueqi_social_and_leisure_infrastructure,
        ueqi_citywide_space,
        median_salary,
        factories_total,
    ]

    # Predict migration using the model
    predicted_migration = model.predict(city_parameters)

    # Return the absolute error (to minimize)
    return round(np.exp(predicted_migration), 2)
    # return abs(target_migration - np.exp(predicted_migration))

In [9]:
model = None
try:
    model = CatBoostRegressor().load_model("city_migr_pred_1711_base.cbm")
    scaler_x = joblib.load("scaler_x.pkl")
except Exception as ex:
    print(ex)


if not model:
    model, X_resampled, X_train, X_test, y_train, y_test, scaler_x = make_regr_model(
        cities
    )
    model.save_model("city_migr_pred_1711_base.cbm")

catboost/libs/model/model_import_interface.h:19: Model file doesn't exist: city_migr_pred_1711_base.cbm


r_index: 100%|##########| 21/21 [00:00<00:00, 1375.78it/s]


0:	learn: 0.6159544	test: 0.4790098	best: 0.4790098 (0)	total: 54.8ms	remaining: 2m 44s
300:	learn: 0.1532917	test: 0.1816715	best: 0.1816715 (300)	total: 329ms	remaining: 2.95s
600:	learn: 0.1090924	test: 0.1692108	best: 0.1692002 (597)	total: 642ms	remaining: 2.56s
900:	learn: 0.0850359	test: 0.1648783	best: 0.1648761 (896)	total: 919ms	remaining: 2.14s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.1628044855
bestIteration = 1078

Shrink model to first 1079 iterations.
0.3624852872782804
0.1677798922017148
0.226225196779196


In [10]:
name = "Ростовская область, Шахты"

In [11]:
cities.loc[cities["region_city"] == name, :]

,region_city,city_category,population,harsh_climate,ueqi_score,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,...,probability_to_move,probability_to_move_to_SELECTED_CITY,migrations_to_selected_city,num_in_migration,one_vacancy_out_response,estimate,h3_index,x,y,geometry
963,"Ростовская область, Шахты",Большой город,222489,False,161,23,26,20,22,30,...,0.278661,0.063002,1.22343,66.530985,5.817,0.741,862d4552fffffff,40.22,47.71,POINT (40.22000 47.71000)


In [12]:
selected_city = cities.loc[cities["region_city"] == name, cols]
x = _preprocess_x(selected_city, scaler_x, fit=False)[0]
x = [round(val, 3) for val in x]


# Use functools.partial to bind extra arguments to the objective function
objective_with_params = partial(
    objective,
    # target_migration=200,  # Example target migration number
    population=x[0],  # Example fixed population
    harsh_climate=x[1],  # Example fixed climate value
    ueqi_residential_current=x[2],  # Example current value
    ueqi_street_networks_current=x[3],
    ueqi_green_spaces_current=x[4],
    ueqi_public_and_business_infrastructure_current=x[5],
    ueqi_social_and_leisure_infrastructure_current=x[6],
    ueqi_citywide_space_current=x[7],
    median_salary_current=x[8],  # Example current salary
    factories_total=x[9],
    model=model,
)

# Set up the Optuna study
study = optuna.create_study(direction="maximize")

# Optimize, passing the pre-bound function
study.optimize(objective_with_params, n_trials=50, n_jobs=1, show_progress_bar=True)

# Get the best parameters
print(
    f"Res: (based on initial o-d data)\nfrom  {round(cities.loc[cities['region_city'] == name,'num_in_migration'].item(),2)}  -- to -->  {study.best_value}\n\nOptimal city parameters: (scaled)\n"
)
_ = [
    print(f"{x[c]}  -- to -->  {round(v,3)} | {k}")
    for c, (k, v) in enumerate(study.best_params.items())
]

  0%|          | 0/50 [00:00<?, ?it/s]

Res: (based on initial o-d data)
from  66.53  -- to -->  95.74

Optimal city parameters: (scaled)

0.249  -- to -->  0.568 | ueqi_residential
0.0  -- to -->  0.377 | ueqi_street_networks
0.256  -- to -->  0.916 | ueqi_green_spaces
0.229  -- to -->  0.736 | ueqi_public_and_business_infrastructure
0.25  -- to -->  0.639 | ueqi_social_and_leisure_infrastructure
0.357  -- to -->  0.815 | ueqi_citywide_space
0.5  -- to -->  0.568 | median_salary
